# 融合检索在文档搜索中的应用

## 概述

此代码实现了一个融合检索系统，该系统结合了基于向量的相似性搜索和基于关键词的BM25检索。这种方法旨在利用两种方法的优势，提高文档检索的整体质量和相关性。

BM25是一种用于信息检索和搜索引擎的排名函数，它基于概率检索模型。BM25是Best Matching 25的缩写，由Stephen Robertson、Karen Spärck Jones、Mandy Walker和Clare Walker于1994年提出的。它是基于BM14算法的改进版本，增加了对文档长度的考虑，以更好地处理不同长度文档的权重分配。

BM25的核心思想是，如果一个查询词在文档中出现的频率较高，并且在整个文档集合中出现的频率较低，那么这个文档与查询词的相关性就更高。BM25考虑了以下两个主要因素：

1. **词频（TF）**：查询词在文档中出现的频率。BM25使用一个非线性函数来减少常见词的权重，以避免它们对排名的影响过大。

2. **逆文档频率（IDF）**：查询词在整个文档集合中出现的频率的倒数。如果一个词在很多文档中都出现，那么它的IDF值会比较低，这意味着它对文档的相关性贡献较小。

BM25的公式如下：

$$\text{BM25} = \sum_{i=1}^{n} \left( \frac{(k_1 + 1) \cdot \text{TF}_{i,d}}{k_1 \cdot (1 - b + b \cdot \frac{\text{l}_d}{\text{avgL}}) + \text{TF}_{i,d}} \right) \cdot \log \frac{(k_3 + 1) \cdot N}{k_3 \cdot \text{DF}_i + N}$$

其中：
- $\text{TF}_{i,d}$ 是查询词 $i$ 在文档 $d$ 中的词频。

- $\text{DF}_i$ 是查询词 $i$ 在文档集合中的文档频率。

- $N$ 是文档集合中的文档总数。

- $\text{l}_d$ 是文档 $d$ 的长度。

- $\text{avgL}$ 是文档集合中所有文档的平均长度。

- $k_1$ 和 $b$ 是控制词频和文档长度影响的参数。

- $k_3$ 是控制文档频率影响的参数

BM25算法广泛应用于现代搜索引擎和文档检索系统中，因其有效性和效率而被广泛认可。


## 动机

传统的检索方法通常依赖于语义理解（基于向量）或关键词匹配（BM25）。每种方法都有其优点和缺点。融合检索旨在将这些方法结合起来，创建一个更加健壮和准确的检索系统，可以有效地处理更广泛的查询。

## 关键组件

1. PDF处理和文本分块
2. 使用FAISS和OpenAI嵌入创建向量存储
3. 为基于关键词的检索创建BM25索引
4. 自定义融合检索函数，结合了两种方法

## 方法细节

### 文档预处理

1. 加载PDF并使用RecursiveCharacterTextSplitter将其分割成块。
2. 通过将't'替换为空白来清理块（可能解决了特定的格式问题）。

### 向量存储创建

1. 使用OpenAI嵌入为文本块创建向量表示。
2. 从这些嵌入创建FAISS向量存储，以实现高效的相似性搜索。

### BM25索引创建

1. 从用于向量存储的相同文本块创建BM25索引。
2. 这允许在基于向量的方法旁边进行基于关键词的检索。

### 融合检索函数

`fusion_retrieval` 函数是此实现的核心：

1. 它接受一个查询并执行基于向量和基于BM25的检索。
2. 来自两种方法的分数被归一化到一个共同的尺度。
3. 计算这些分数的加权组合（由`alpha`参数控制）。
4. 根据组合分数对文档进行排名，并返回前k个结果。

## 这种方法的好处

1. 改进的检索质量：通过结合语义和基于关键词的搜索，系统可以捕捉概念相似性和精确的关键词匹配。
2. 灵活性：`alpha`参数允许根据特定用例或查询类型调整向量和关键词搜索之间的平衡。
3. 健壮性：组合方法可以有效地处理更广泛的查询，减轻了个别方法的弱点。
4. 可定制性：系统可以轻松适应使用不同的向量存储或基于关键词的检索方法。

## 结论

融合检索代表了一种强大的文档搜索方法，它结合了语义理解和关键词匹配的优势。通过利用基于向量和BM25的检索方法，它为信息检索任务提供了一个更全面和灵活的解决方案。这种方法在许多领域都有潜在的应用，其中概念相似性和关键词相关性都很重要，例如学术研究、法律文档搜索或通用搜索引擎。


<div style="text-align: center;">

<img src="../images/fusion_retrieval.svg" alt="Fusion Retrieval" style="width:100%; height:auto;">
</div>

### Import libraries 

In [2]:
import os
import sys
from dotenv import load_dotenv
from langchain.docstore.document import Document
# Load environment variables from a .env file
load_dotenv()

from typing import List
from rank_bm25 import BM25Okapi
import numpy as np


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from rag.helper_functions import *
from rag.evaluation.evalute_rag import *

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Define document path

In [3]:
path = "../data/Understanding_Climate_Change.pdf"

### 将PDF编码为向量存储，并返回上一步拆分的文档以创建BM25实例。

In [4]:
def encode_pdf_and_get_split_documents(path, chunk_size=1000, chunk_overlap=200):
    """
    将PDF书籍使用OpenAI嵌入编码到向量存储中。

    参数：
        path: PDF文件的路径。
        chunk_size: 每个文本块的期望大小。
        chunk_overlap: 连续块之间的重叠量。

    返回：
        包含编码书籍内容的FAISS向量存储。
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore, cleaned_texts

### 创建向量存储并获取分块文档。

In [5]:
vectorstore, cleaned_texts = encode_pdf_and_get_split_documents(path)

### 创建一个BM25索引以通过关键词检索文档。

In [6]:
def create_bm25_index(documents: List[Document]) -> BM25Okapi:
    """
    创建一个来自给定文档的BM25索引。

    BM25（Best Matching 25）是一种用于信息检索的排名函数。
    它基于概率检索框架，是对TF-IDF的改进。

    参数：
    documents (List[Document]): 要索引的文档列表。

    返回：
    BM25Okapi: 一个可用于BM25评分的索引。
    """
    # 通过在空白处分割来对每个文档进行分词
    # 这是一种简单的方法，可以通过更复杂的分词技术进行改进
    tokenized_docs = [doc.page_content.split() for doc in documents]
    return BM25Okapi(tokenized_docs)

In [7]:
bm25 = create_bm25_index(cleaned_texts) # Create BM25 index from the cleaned texts (chunks)

### 定义一个函数，该函数通过语义和关键词检索，标准化分数，并获取前k个文档

In [8]:
def fusion_retrieval(vectorstore, bm25, query: str, k: int = 5, alpha: float = 0.5) -> List[Document]:
    """
    执行融合检索，结合基于关键词（BM25）和基于向量的搜索。

    参数：
    vectorstore (VectorStore): 包含文档的向量存储。
    bm25 (BM25Okapi): 预先计算的BM25索引。
    query (str): 查询字符串。
    k (int): 要检索的文档数量。
    alpha (float): 向量搜索分数的权重（1-alpha将是BM25分数的权重）。

    返回：
    List[Document]: 根据组合分数排序的前k个文档。
    """
    # Step 1: 从向量存储中得到所有的文档
    all_docs = vectorstore.similarity_search("", k=vectorstore.index.ntotal)

    # Step 2: 实施 BM25 搜索
    bm25_scores = bm25.get_scores(query.split())

    # Step 3: 实施向量搜索
    vector_results = vectorstore.similarity_search_with_score(query, k=len(all_docs))
    
    # Step 4: 归一化分数
    vector_scores = np.array([score for _, score in vector_results])
    vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores))

    bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores))

    # Step 5: 加权
    combined_scores = alpha * vector_scores + (1 - alpha) * bm25_scores  

    # Step 6: 根据加权后的分数重排文档
    sorted_indices = np.argsort(combined_scores)[::-1]
    
    # Step 7: 返回分数最大的前k个
    return [all_docs[i] for i in sorted_indices[:k]]

### Use Case example

In [9]:
# Query
query = "What are the impacts of climate change on the environment?"

# Perform fusion retrieval
top_docs = fusion_retrieval(vectorstore, bm25, query, k=5, alpha=0.5)
docs_content = [doc.page_content for doc in top_docs]
show_context(docs_content)

Context 1:
workshops, and community events. Lifelong learning fosters a culture of con tinuous 
improvement and adaptability.  
Intergenerational Dialogue  
Youth Engagement  
Engaging youth in climate action is critical for long -term sustainability. Youth bring energy, 
creativity, and a sense of urgency to climate movements. Providing platforms for youth 
voices, supporting youth -led initiatives, and involving young people in de cision -making 
processes are essential for meaningful engagement.  
Intergenerational Collaboration  
Intergenerational collaboration involves working together across age groups to address 
climate challenges. This includes mentorship programs, intergenerational projects, and 
dialogue forums. Sharing knowledge and experiences between generations enhances 
collective capacity and resilience.


Context 2:
This vision includes a healthy planet, thriving ecosystems, and equitable societies. Working 
together towards this vision creates a sense of purpose and 